In [1]:
# Importaciones necesarias
import mlflow.pyfunc
from datetime import timedelta
import pandas as pd
import dagshub
import yfinance as yf

In [2]:
# Definir la configuración del modelo y el alias del modelo optimizado
dagshub.init(url="https://dagshub.com/PacoTinoco/Proyecto_Final_CDD", mlflow=True)
MLFLOW_TRACKING_URI = mlflow.get_tracking_uri()

model_name = "google-stock-model-randomforest-perfect"
alias = "champion"
model_uri = f"models:/{model_name}@{alias}"

# Cargar el modelo entrenado de MLflow
model = mlflow.pyfunc.load_model(model_uri)


Accessing as PacoTinoco

Initialized MLflow to track repo "PacoTinoco/Proyecto_Final_CDD"

Repository PacoTinoco/Proyecto_Final_CDD initialized!

In [23]:
# Solicitar datos al usuario
capital_inicial = float(input("Ingrese el capital inicial a invertir en USD: "))
dias_inversion = int(input("Ingrese el número de días de inversión: "))


In [24]:
# Descargar datos históricos recientes para Google
data = yf.download('GOOGL', period="1mo")
# Usar la última fila como base
input_data = data.iloc[-1:]

# Iterar para simular la predicción de cada día adicional
precio_inicial = input_data['Close']
cantidad_acciones = capital_inicial / precio_inicial
precio_estimado = precio_inicial


[*********************100%***********************]  1 of 1 completed


In [25]:
input_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-11-14,178.279999,178.820007,174.320007,175.580002,175.580002,30989100


In [26]:
precio_inicial = input_data['Close']
precio_inicial

Date
2024-11-14    175.580002
Name: Close, dtype: float64

In [27]:
for _ in range(dias_inversion):
    # Predecir el precio de cierre del día
    precio_estimado = model.predict(input_data)
    # Actualizar el dataframe de entrada con el precio proyectado
    input_data['Close'] = precio_estimado
    input_data.index += timedelta(days=1)

# Calcular el capital final con la proyección de precio final
capital_final = cantidad_acciones * precio_estimado
ganancia_estimada = capital_final - capital_inicial

C:\Users\Francisco\AppData\Local\Temp\ipykernel_10600\614707743.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_data['Close'] = precio_estimado


In [28]:
ganancia_estimada    

Date
2024-11-14   -150.232603
Name: Close, dtype: float64

In [29]:
# Extraer valores numéricos para evitar el error de Series
capital_final = capital_final.iloc[0] if isinstance(capital_final, pd.Series) else capital_final
ganancia_estimada = ganancia_estimada.iloc[0] if isinstance(ganancia_estimada, pd.Series) else ganancia_estimada

In [31]:
# Mostrar el resultado
print(f"Se estima que tendrás aproximadamente ${capital_final:.2f} USD al final de {dias_inversion} días, con una ganancia de ${ganancia_estimada:.2f} USD.")

Se estima que tendrás aproximadamente $849.77 USD al final de 1 días, con una ganancia de $-150.23 USD.
